In [1084]:
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as cst

<span style="font-size:40px;">**Niveau 1**</span>


In [1085]:
class Maillage : 
    #Initialisation de l'objet.
    def __init__(self,X : np.ndarray ,Y : np.ndarray,Z : np.ndarray) :         
        if not((size := X.size) == Y.size == Z.size) : 
            raise ValueError('Les 3 matrices doivent avoir les mêmes dimensions.')
        
        #Les listes sont flatten par convénience.
        X,Y,Z = X.flatten(),Y.flatten(),Z.flatten()
        self.X = X
        self.Y = Y
        self.Z = Z
        self.XYZ = np.array([X,Y,Z])
        self.nb_points = size
        
        #-- ces deux la ne sont plus utile
        self.nb_points_droite = np.cbrt(size)
        self.nb_points_plan = self.nb_points_droite**2
        #--
    
    #Définition de l'addition de "Maillage1 + Maillage2".
    def __add__(self,maillage : "Maillage") -> "Maillage" : 
        X,Y,Z = np.concatenate((self.X,maillage.X)),np.concatenate((self.Y,maillage.Y)),np.concatenate((self.Z,maillage.Z))
        return  Maillage(X,Y,Z) 
    
    #Définition de la multiplication scalaire ie "scalaire * Maillage".
    def __rmul__(self,scalaire) :
        return Maillage(scalaire*self.X,scalaire*self.Y,scalaire*self.Z) 
    
    #Définition de l'expression Maillage[indice].
    def __getitem__(self,index : int) -> "Maillage" : 
        return self.X[index],self.Y[index],self.Z[index]
    
    #Définition de la suppression d'un item du maillage. ie "del Maillage[indice]".
    #on la supprimera surement si on ne l'utilise pas 
    def __delitem__(self,index : int) : 
        X = np.concatenate((self.X[:index], self.X[index+1:]))
        Y = np.concatenate((self.Y[:index], self.Y[index+1:]))
        Z = np.concatenate((self.Z[:index], self.Z[index+1:]))
        
        maillage = Maillage(X,Y,Z)
        
        for attr in list(self.__dict__.keys()) : 
            setattr(self,attr, getattr(maillage,attr))
    
    #Définition de l'égalité entre deux maillages ie "Maillage1 == Maillage2".
    def __eq__(self, maillage : "Maillage"):
        return np.array_equal(self.X, maillage.X) and \
               np.array_equal(self.Y, maillage.Y) and \
               np.array_equal(self.Z, maillage.Z)


    def translate(self, dl : tuple[float]) -> "Maillage":
        if len(dl) != 3:
            raise ValueError('Trop ou pas assez de translations ont été données.')

        return Maillage(self.X + dl[0], self.Y + dl[1], self.Z + dl[2])
    
    def norm(self) : 
        return np.sqrt(self.X**2 + self.Y**2 + self.Z**2)
    
    def rotate(self, axes : list[str], angles : list[float]) -> "Maillage" : 
        if len(axes) != len(angles):
            raise ValueError("Trop ou pas assez d'angles et/ou axes ont été donnés.")

        Rx = lambda angle : np.array([[1,0,0],[0,np.cos(angle),-np.sin(angle)],[0,np.sin(angle),np.cos(angle)]])
        Ry = lambda angle : np.array([[np.cos(angle),0,np.sin(angle)],[0,1,0],[-np.sin(angle),0,np.cos(angle)]])
        Rz = lambda angle : np.array([[np.cos(angle),-np.sin(angle),0],[np.sin(angle),np.cos(angle),0],[0,0,1]])
        product = ''
    
        for axe,angle in zip(axes,angles) : 
            product = f'R{axe.strip().lower()}({angle}) @' + product
        
        return Maillage(*eval(f'{product} self.XYZ')) 
    


In [1086]:
def cylindrique_to_cartesien(r : float, theta : float, z : float) -> tuple:
    """
    Entrées :
        - r: valeur du rayon (distance entre le point et son projeté orthogonal sur l'axe du cylindre).
        - theta: valeur de l’angle.
        - z: valeur de l'altitude.

    Sortie :
        - Renvoie un couple (x,y,z) représentant les coordonnées du point en cartésien.

    Explication:
        - Prend des coordonnées cylindriques et les traduit en coordonnées cartésiennes.
    """
    return r*np.cos(theta), r*np.sin(theta), z

In [1087]:
def spherique_to_cartesien(r : float, theta : float, phi : float) -> tuple:
    """
    Entrées :
        - r : valeur du rayon.
        - theta : valeur de l’angle entre OM (M point quelconque de l’espace) et Oz.
        - phi : valeur de l’angle entre le projeté orthogonal de OM sur le plan (xOy) et l’axe Ox.
    
    Sortie :
        - Renvoie un couple (x,y,z) représentant les coordonnées du point en cartésien.

    Explication:
        - Prend des coordonnées sphériques et les traduit en coordonnées cartésiennes.
    """
    return r * np.cos(phi) * np.sin(theta), r * np.sin(phi) * np.sin(theta), r * np.cos(theta)

<span style="font-size:40px;">**Niveau 2**</span>


In [1088]:
'''def creer_maillage_cartesien(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> Maillage:
    """
    Entrées:

        - x_min : la valeur minimale en x du maillage (Exemple : x_min = -2.5). Le signe est quelconque et la valeur doit être différente de x_max.
        - y_min : la valeur minimale en y du maillage (Exemple : y_min = -2.5). Le signe est quelconque et la valeur doit être différente de y_max.
        - z_min : la valeur minimale en z du maillage (Exemple : z_min = -2.5). Le signe est quelconque et la valeur doit être différente de z_max.

        - x_max : la valeur maximale en x du maillage (Exemple : x_max = 2.5). Le signe est quelconque et la valeur doit être différente de x_min.
        - y_max : la valeur maximale en y du maillage (Exemple : y_max = 2.5). Le signe est quelconque et la valeur doit être différente de y_min.
        - z_max : la valeur maximale en z du maillage (Exemple : z_max = 2.5). Le signe est quelconque et la valeur doit être différente de z_min.

        - pas_du_maillage : 
            
            * Distance séparant 2 points (non diagonnaux) du maillage. 
            
            * Il est de préférence un diviseur de (x_max - x_min) et de (y_max - y_min) et (z_max - z_min). 
            
            * Éviter un pas égal à 0.3 ou quelconque multiple de 0.3 ainsi que les nombres premiers par exemple. 
            
            * Sa valeur ne doit pas être plus élevée que la moitié de la plus petite distance coordonnée_max - coordonnée_min des 3 axes.
        
    Sorties:
    
        - Maillage(X,Y,Z)
            
    Explication:
    
        La fonction creer_maillage_cartesien() crée une maillage de l'espace pour des limites et un pas donnés.
    """
    
    est_un_plan = False
    est_une_droite = False
    #Vérification de la cohérence des données.
    for min, max in zip([x_min,y_min,z_min],[x_max,y_max,z_max]):
        min_max_verification = True
        if min == max: #Sinon le maillage est soit: un plan, une droite ou un point ce qui ne nous intéresse pas ici.
            if est_un_plan:
                if est_une_droite:
                    raise ValueError("Le maillage est un point.") #L'intérêt de créer un maillage ponctuel est nul, autant calculer les valeurs directement au point.
                est_une_droite = True
                min_max_verification = False
            est_un_plan = True    
            min_max_verification = False
    
        if min > max:
            min, max = max, min #Nécessaire pour avoir des np.linspace(min, max, nb_points) cohérents.
        
        if min_max_verification:   
            if pas_du_maillage > abs(max-min)/2:
                raise ValueError("pas_du_maillage est trop grand.") #Le cas écheant, le maillage ne serait pas bien défini.

    #Création du maillage.
    longueur_maillage_x = abs(x_max - x_min)
    longueur_maillage_y = abs(y_max - y_min)
    longueur_maillage_z = abs(z_max - z_min)
    
    if pas_du_maillage == 0 : raise ZeroDivisionError
    
    nb_points_entre_x_min_x_max = int((longueur_maillage_x) / (pas_du_maillage)) + 1
    nb_points_entre_y_min_y_max = int((longueur_maillage_y) / (pas_du_maillage)) + 1
    nb_points_entre_z_min_z_max = int((longueur_maillage_z) / (pas_du_maillage)) + 1
    
    
    axe_x = np.linspace(x_min, x_max, nb_points_entre_x_min_x_max)
    axe_y = np.linspace(y_min, y_max, nb_points_entre_y_min_y_max)
    axe_z = np.linspace(z_min, z_max, nb_points_entre_z_min_z_max)
    
    X,Y,Z = np.meshgrid(axe_x,axe_y,axe_z)
    
    #Permet de gérer les cas où pas_du_maillage engendre des points dont les coordonnées ont des décimales infinies comme 0.3.
    X = np.round(X, 1) 
    Y = np.round(Y, 1)
    Z = np.round(Z, 1)

    return Maillage(X,Y,Z)'''

'def creer_maillage_cartesien(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> Maillage:\n    """\n    Entrées:\n\n        - x_min : la valeur minimale en x du maillage (Exemple : x_min = -2.5). Le signe est quelconque et la valeur doit être différente de x_max.\n        - y_min : la valeur minimale en y du maillage (Exemple : y_min = -2.5). Le signe est quelconque et la valeur doit être différente de y_max.\n        - z_min : la valeur minimale en z du maillage (Exemple : z_min = -2.5). Le signe est quelconque et la valeur doit être différente de z_max.\n\n        - x_max : la valeur maximale en x du maillage (Exemple : x_max = 2.5). Le signe est quelconque et la valeur doit être différente de x_min.\n        - y_max : la valeur maximale en y du maillage (Exemple : y_max = 2.5). Le signe est quelconque et la valeur doit être différente de y_min.\n        - z_max : la valeur maximale en z du maillage (Exemple : z_max 

In [1089]:
def creer_maillage_plan(x_min : float, x_max : float, y_min : float, y_max : float, z : "function", pas_x : float, pas_y : float) -> Maillage : 
    densite_x = int(1/pas_x)
    densite_y = int(1/pas_y)
    
    x = np.linspace(x_min,x_max, densite_x*int(x_max - x_min)+1)
    y = np.linspace(y_min,y_max, densite_y*int(y_max - y_min)+1)
    X,Y = np.meshgrid(x,y)
    Z = np.full_like(X, z(X, Y))
    
    return Maillage(X,Y,Z)

In [1090]:
def creer_maillage_cartesien(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> Maillage : 
    M = Maillage(*np.empty((3,0)))
    densite_x = int(1/pas_du_maillage)
    Z = np.linspace(z_min,z_max, densite_x*int(z_max - z_min)+1)
    
    for z in Z : 
        M += creer_maillage_plan(x_min,x_max,y_min,y_max,lambda x,y : z, pas_du_maillage,pas_du_maillage)
    
    return M

In [1091]:
def creer_maillage_cylindrique(z_min : float, z_max : float, rayon : "function", theta : float, pas_z : float, pas_theta : float) -> Maillage :
    """
    Entrées :

        - z_min : Représente la valeur minimale selon z.
        - z_max : Représente la valeur maximale selon z.
            - NB : Z_max – Z_min = hauteur.

        - rayon : C’est une fonction, elle dépend de theta et de z.
        - theta : représente l’angle d’ouverture ducylindre.

        - pas_du_maillage :

            * Distance séparant 2 points (non diagonnaux) du maillage.

            * Il est de préférence un diviseur de (theta_max - theta_min) et de(rayon_max - rayon_min) et (z_max - z_min).

            * Éviter un pas égal à 0.3 ou quelconque multiple de 0.3 ainsi que les nombres premiers par exemple.

            * Sa valeur ne doit pas être plus élevée que la moitié de la plus petite distance coordonnée_max - coordonnée_min des 3 axes.

    - Sorties :
    
        - Renvoie un maillage qui représente un cylindre ou un morceau de cylindre en fonction de l’angle annoncé.
    """

    #je ne sais pas si ces conditions sont utiles
    if z_min > z_max:
        z_min, z_max = z_max, z_min
    if theta > 2 * np.pi:
        theta = 2 * np.pi

    densite_theta = int(1/pas_theta)
    densite_z = int(1/pas_z)

    valeurs_theta = np.linspace(0, theta, densite_theta * int(theta),endpoint=False) 
    valeurs_z = np.linspace(z_min, z_max, densite_z*int(z_max - z_min) + 1,endpoint=False)

    THETA, Z = np.meshgrid(valeurs_theta, valeurs_z)
    array_r = np.full_like(THETA,rayon(THETA,Z))

    return Maillage(*cylindrique_to_cartesien(array_r, THETA, Z))

In [1092]:
def creer_maillage_spherique(rayon : float, theta : float, phi : float, pas_phi : float, pas_theta) -> Maillage :
    densite_phi = int(1/pas_phi)
    densite_theta = int(1/pas_theta)
    
    Theta = np.linspace(0, theta, densite_theta * int(theta),endpoint=False) 
    Phi =  np.linspace(0, phi, densite_phi * int(phi),endpoint=False) 

    THETA, PHI = np.meshgrid(Theta,Phi)
    RAYON = np.full_like(THETA,rayon(THETA,PHI))
    
    return Maillage(*spherique_to_cartesien(RAYON,THETA,PHI))

In [1093]:
def creer_maillage_disque(rayon : float, theta : float, z : "function", pas_rayon : float, pas_theta : float) -> Maillage : 
    densite_rayon = int(1/pas_rayon)
    densite_theta = int(1/pas_theta)
    
    Theta = np.linspace(0, theta, densite_theta * int(theta),endpoint=False) 
    Rayon =  np.linspace(0, rayon, densite_rayon * int(rayon)) 

    THETA, RAYON = np.meshgrid(Theta,Rayon)
    Z = np.full_like(THETA,z(THETA,RAYON))
    
    return Maillage(*cylindrique_to_cartesien(RAYON, THETA, Z))

<span style="font-size:40px;">**Niveau 3**</span>


In [1094]:
def calcule_charge(maillage: Maillage, distribution_charge : tuple[float,"function"] = None) : 
    return (val := distribution_charge[1](*maillage.XYZ))*distribution_charge[0]/np.sum(val) if distribution_charge != None else None
    
def afficher_plotly(maillage : Maillage, distribution_charge : tuple[float,"function"] = None) : 
    fig = go.Figure(go.Scatter3d(x=maillage.X,y=maillage.Y,z=maillage.Z,mode="markers",
                                 marker=dict(color=calcule_charge(maillage,distribution_charge),colorscale="Viridis",showscale=True if distribution_charge != None else False,size=1)))
    fig.show()

def afficher_matplotlib(maillage : Maillage, distribution_charge : tuple[float,"function"] = None) :

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(maillage.X, maillage.Y, maillage.Z, marker='o', c = calcule_charge(maillage,distribution_charge), cmap="coolwarm", s = 0.5)

    if distribution_charge is not None:
        cbar = fig.colorbar(sc, ax=ax, shrink=0.5)
        cbar.set_label("valeurs")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    plt.show()

In [1095]:
'''def isoler_plan_cartesien(maillage : Maillage, plan : str, valeur_coupe : float) -> tuple:
  
    val_min_x, val_min_y, val_min_z = maillage[0]
    val_max_x, val_max_y, val_max_z = maillage[maillage.nb_points - 1]
    
    pas = round((val_max_x - val_min_x) / maillage.nb_points_droite, 2)
    
    nb_plan = int(maillage.nb_points / maillage.nb_points_plan)
    
    if valeur_coupe == None:
            raise ValueError('Précisez la valeur de la côte du plan.')
    
    if (psl := plan.strip().lower()) in ["xoz","zox"]:
        if val_min_y > valeur_coupe > val_max_y and (valeur_coupe not in maillage.Y) :
            raise ValueError("Le plan n'existe pas.")
        val_indice_y = int(abs(val_min_y - valeur_coupe) / pas * maillage.nb_points_plan)
        
        return maillage[val_indice_y:val_indice_y+int(maillage.nb_points_plan)]
    
    if psl in ["xoy","yox"]:
        if val_min_z > valeur_coupe > val_max_z and (valeur_coupe not in maillage.Z) :
            raise ValueError("Le plan n'existe pas.")
        val_indice_z = int(abs(val_min_z - valeur_coupe) / pas)
        
        liste = [float(valeur) for valeur in maillage.Y[::int(maillage.nb_points_plan)]]
        
        return maillage.X[:int(maillage.nb_points_plan)], liste*nb_plan, maillage.Z[val_indice_z::int(maillage.nb_points_droite)]
    
    if psl in ["yoz","zoy"]:
        if val_min_x > valeur_coupe > val_max_x  and (valeur_coupe not in maillage.X) :
            raise ValueError("Le plan n'existe pas.")
        
        liste_x = [float(valeur_coupe) for _ in range(int(maillage.nb_points_plan))]
        liste_y = [[float(valeur)]*nb_plan for valeur in maillage.Y[::int(maillage.nb_points_plan)]]
    
        return liste_x, liste_y, maillage.Z[:int(maillage.nb_points_plan)]
    
    raise ValueError("La string du plan ne convient pas. Vérifiez l'orthographe ou si le plan existe.")'''

'def isoler_plan_cartesien(maillage : Maillage, plan : str, valeur_coupe : float) -> tuple:\n  \n    val_min_x, val_min_y, val_min_z = maillage[0]\n    val_max_x, val_max_y, val_max_z = maillage[maillage.nb_points - 1]\n    \n    pas = round((val_max_x - val_min_x) / maillage.nb_points_droite, 2)\n    \n    nb_plan = int(maillage.nb_points / maillage.nb_points_plan)\n    \n    if valeur_coupe == None:\n            raise ValueError(\'Précisez la valeur de la côte du plan.\')\n    \n    if (psl := plan.strip().lower()) in ["xoz","zox"]:\n        if val_min_y > valeur_coupe > val_max_y and (valeur_coupe not in maillage.Y) :\n            raise ValueError("Le plan n\'existe pas.")\n        val_indice_y = int(abs(val_min_y - valeur_coupe) / pas * maillage.nb_points_plan)\n        \n        return maillage[val_indice_y:val_indice_y+int(maillage.nb_points_plan)]\n    \n    if psl in ["xoy","yox"]:\n        if val_min_z > valeur_coupe > val_max_z and (valeur_coupe not in maillage.Z) :\n        

In [1110]:
x_min,x_max,y_min,y_max,z_min,z_max = -2,2,-3,4,0,5
pas_du_maillage = 0.5
Maillage_espace = creer_maillage_cartesien(x_min,x_max,y_min,y_max,z_min,z_max,pas_du_maillage)

In [1133]:
def isoler_plan_cartesien(M : Maillage, plan : str, valeur_coupe : float) -> Maillage :
    plan = ''.join(sorted(plan.strip().lower()))
    index = {'yz' : 0,'xz' : 1,'xy' : 2}
    mask = M.XYZ[index[plan],:] == valeur_coupe
    if np.any(mask) == False : raise ValueError("la valeur de coupe choisit n'est pas autorisée")
    return Maillage(* M.XYZ[:,mask]) 
    
afficher_plotly(isoler_plan_cartesien(Maillage_espace, 'xz',0))

In [1098]:
#M2 = creer_maillage_cylindrique(-2,2,lambda theta,z : 2,2*np.pi, 0.01,0.01)
#M1 = creer_maillage_spherique(lambda theta,phi : phi, np.pi,2*np.pi, 0.1,0.1) #creer_maillage_cylindrique(-3,3,lambda theta,z : 2,2*np.pi, 0.05,0.05) #np.where(abs(z) >= 1/2, z, 2*z)  np.where(abs(z) <= 1, z+theta, z)
#M3 = creer_maillage_plan(-2,2,-2,2,lambda x,y : np.cos(x*y),0.05,0.05)
#afficher_plotly(M3,(10,lambda x,y,z : abs(x)))

In [1108]:
rayon = 2
theta = 2*np.pi
pas_rayon = pas_z = 0.1
pas_theta = 0.1
C = creer_maillage_cylindrique(-2,2,lambda theta,z : 2 ,theta,pas_z,pas_theta)
B1 = creer_maillage_disque(rayon,theta,lambda theta,r : 2,pas_rayon,pas_theta)
B2 = creer_maillage_disque(rayon,theta,lambda theta,r : -2,pas_rayon,pas_theta)

In [1109]:
def calcule_champ(maillage : Maillage, point : np.ndarray, charge : np.ndarray) : 
    k= 1/(4*np.pi*cst.epsilon_0)
    M = (-1*maillage).translate(point)
    return np.sum(k*charge*M.XYZ/(M.norm()**3), axis=1)

M = C+B1+B2
distribution_charge = (10,lambda x,y,z : 1)
point = np.array([2,2,2])
valeurs_charge = calcule_charge(M,distribution_charge)
x,y,z = 10**(-15)*calcule_champ(M,point,valeurs_charge)

fig = go.Figure(go.Scatter3d(
    x=M.X,y=M.Y,z=M.Z,mode="markers",
    marker=dict(color=valeurs_charge,colorscale="Viridis",showscale=True if distribution_charge != None else False,size=1)))

fig.add_traces(data=go.Scatter3d(
    x=[point[0], x],
    y=[point[1], y],
    z=[point[2], z],
    mode='lines+markers',
    marker=dict(size=5, color='red'),
    line=dict(color='red', width=4)
))

fig.show()